In [5]:
import torch
from torch.optim import Adam
import torch.nn as nn
import torchvision 
from PIL import Image

In [ ]:
# (1)DEFINE MODEL STEPS IN ALGORITHM in INIT and SUPER
# (2)DEFINE ACTUAL MODEL PASS in FORWARD
# (3)SEND MODEL TO DEVICE
# (4)DEFINE LOSS and OPTIMIZATION 
# (5)LOOP training data 
# (6)Image Classifier Neural Network 
#  loop for each image (X) and label(y) in dataset 
#  i)img =as it takes last layer o/p as i/p here reshape to the o/p to batch size, actual size and send to device
#  ii)label (yhat) =each label send to the device
#  ``````````for FORWARD FLOW```````````
#  iii)output =now send img in model to train
#  iv) calculate loss by yhat - y
#  `````````for BACKWARD FLOW```````````` 
#  v) set optimizer to zero_grad 
#  vi) loss call
#  vii) optimizer call

In [13]:
#HyPERLINK DECLARATIONS
batch_size = 100
num_epochs = 2

#GETTING DATA
train_data = torchvision.datasets.MNIST(root="../data",train=True,transform = torchvision.transforms.ToTensor())
test_data = torchvision.datasets.MNIST(root="../data",train=False,transform = torchvision.transforms.ToTensor())

#DATA LOADING
train_load = torch.utils.data.DataLoader(train_data,shuffle=True,batch_size=batch_size)
test_load = torch.utils.data.DataLoader(test_data,shuffle=True,batch_size=batch_size)

class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier,self).__init__()
        self.model = nn.Sequential(
                        nn.Conv2d(1,32,(3,3)),
                        nn.ReLU(),
                        nn.Conv2d(32,64,(3,3)),
                        nn.ReLU(),
                        nn.Conv2d(64,64,(3,3)),
                        nn.ReLU(),
                        nn.Flatten(),
                        nn.Linear(64*(28-6)*(28-6),10)) #28-6 becoz 3 twice from 2 layers=6
    def forward(self, x): 
        return self.model(x)

model = ImageClassifier().to('cpu')

#LOSS and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr= 0.001)

#TAINING FLOW
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_load):
        img = images.reshape(batch_size,1, 28,28).to('cpu')
        labels = labels.to('cpu')
        
        # Forward pass
        outputs = model(img)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}],Loss: {loss.item():.4f}')

Epoch [1/2],Loss: 0.1438
Epoch [1/2],Loss: 0.0757
Epoch [1/2],Loss: 0.0607
Epoch [1/2],Loss: 0.0532
Epoch [1/2],Loss: 0.0798
Epoch [1/2],Loss: 0.1042
Epoch [2/2],Loss: 0.0513
Epoch [2/2],Loss: 0.0375
Epoch [2/2],Loss: 0.0602
Epoch [2/2],Loss: 0.0675
Epoch [2/2],Loss: 0.0554
Epoch [2/2],Loss: 0.0304


In [14]:
# Save the model's state_dict directly
torch.save(model.state_dict(), 'model_state.pt')

In [15]:
model.load_state_dict(torch.load('model_state.pt'))
model.eval()  # Set the model to evaluation mode (important for inference)

C:\Users\user\AppData\Local\Temp\ipykernel_1976\845671044.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_state.pt'))


ImageClassifier(
  (model): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=30976, out_features=10, bias=True)
  )
)

In [16]:
# 4. Process and predict on an external 
from torchvision.transforms import transforms
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure image is grayscale
    transforms.Resize((28, 28)),  # Resize to the expected input size (e.g., 28x28 for MNIST-like models)
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize to the same range as training data (you can modify if needed)
])

In [34]:
img = Image.open('test_image/img2.jpg') 
img_tensor = transform(img).unsqueeze(0).to('cpu')

In [35]:
# Make the prediction
with torch.no_grad():  # Disable gradient calculation during inference
    outputs = model(img_tensor)

# Get the predicted class label
_, predicted = torch.max(outputs, 1)

# Print the prediction
print(f"Predicted Label: {predicted.item()}")

Predicted Label: 2
